In [ ]:
import os 
from typing import List, TypedDict
from mistralai import Mistral
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain import hub
from langgraph.graph import START, StateGraph
from langchain_text_splitters import RecursiveCharacterTextSplitter




In [57]:

load_dotenv()

tracing = os.getenv("LANGSMITH_TRACING")
api_key = os.getenv("LANGSMITH_API_KEY")
Mistral_api_key = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

# Chemin vers le dossier contenant vos documents
DOCUMENTS_PATH = "../data"

In [40]:
# Chat model mistral
llm = init_chat_model("mistral-large-latest", model_provider="mistralai")



In [ ]:
# Charger les documents depuis le dossier "data"

loader = DirectoryLoader(
    DOCUMENTS_PATH,
    glob="*.txt",  # Charger uniquement les fichiers .txt
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}  # Spécifier l'encodage approprié
)
docs = loader.load()

if docs:
    print(f"{len(docs)} documents chargés avec succès.")
    for i, doc in enumerate(docs[:5]):  # Afficher les 5 premiers documents
        # Extraire le nom du fichier depuis la métadonnée "source"
        source = doc.metadata.get("source", "inconnu")
        file_title = os.path.basename(source)        
        print(f"Document {i+1}:")
        print(f"Nom du fichier : {file_title}")
        print("-" * 50)
else:
    print("Aucun document n'a été chargé.")

2 documents chargés avec succès.
Document 1:
Nom du fichier : FAQ_kap_numerique.txt
--------------------------------------------------
Document 2:
Nom du fichier : Fiche_action_1.2.5_kap_numerique.txt
--------------------------------------------------


In [42]:
# Découper les documents en chunks pour un meilleur traitement par le modèle
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [43]:
# Initialiser les embeddings Mistral
embeddings = MistralAIEmbeddings()

c:\Python312\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [44]:
#Vector 
vector_store = InMemoryVectorStore(embeddings)
# Créer le vectorstore en mémoire à partir des chunks et des embeddings
vector_store = InMemoryVectorStore.from_documents(all_splits, embeddings)

In [55]:
# Définir le prompt pour la recherche de réponse
prompt = hub.pull("rlm/rag-prompt")

# Définition de l'état de l'application
class State(TypedDict):
    question: str
    context: List  # Liste d'objets Document
    answer: str
    
# Fonction de récupération (retrieval) des documents pertinents
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"], k=3)
    return {"context": retrieved_docs}


# Fonction de génération de la réponse
def generate(state: State):
    # Agrégation du contenu des documents récupérés avec détails sur les sources
    docs_details = []
    for doc in state["context"]:
        source = doc.metadata.get("source", "Source inconnue")
        file_name = os.path.basename(source)
        
        # Extraction des métadonnées : section, page, date de mise à jour
        section = doc.metadata.get("section", "Section non spécifiée")
        page = doc.metadata.get("page", "Page non spécifiée")
        update_date = doc.metadata.get("update_date", "Date non disponible")
        
        docs_details.append({
            "content": doc.page_content,
            "file_name": file_name,
            "section": section,
            "page": page,
            "update_date": update_date
        })
    
    # Agrégation du contenu des documents
    docs_content = "\n\n".join(doc["content"] for doc in docs_details)
    
    # Formatage des sources avec section, page et date
    formatted_sources = "\n".join([
        f"[Document: {doc['file_name']}, Section: {doc['section']}, Page: {doc['page']}, Mise à jour: {doc['update_date']}]" 
        for doc in docs_details
    ])
    
    # Instructions système mises à jour pour inclure la question dans la réponse
    system_instructions = (
        "Tu es un instructeur expert du dispositif KAP Numérique. Tu réponds à des questions en te basant uniquement sur les documents officiels fournis.\n\n"
        "Consignes de réponse :\n"
        "1. Commence ta réponse en répétant la question posée, par exemple : 'En réponse à votre question : \"[question]\", voici les informations demandées :'\n"
        "2. Fournis une réponse concise et structurée.\n"
        "3. Utilise des listes à puces pour organiser les informations.\n"
        "4. Cites systématiquement les sources avec le format suivant : [Document: Nom du document, Section: Nom de la section, Page: Numéro de page, Mise à jour: Date].\n\n"
        "Exemple de réponse attendue :\n"
        "Question: Quels types de projets peuvent être financés par KAP Numérique ?\n\n"
        "Réponse:\n"
        "En réponse à votre question : \"Quels types de projets peuvent être financés par KAP Numérique ?\", voici les informations demandées :\n"
        "- Projets de visibilité numérique et services aux usagers :\n"
        "  - Digitalisation de contenus (logo, charte graphique)\n"
        "  - Création ou refonte d’un site internet ou d’une solution de vente en ligne\n"
        "  - Référencement naturel (SEO)\n"
        "  - Abonnement à une place de marché\n"
        "  - Développement de la présence sur les réseaux sociaux\n"
        "  - Développement d’une application mobile\n"
        "  - Mise en place de chatbots\n"
        "- Projets de sécurité informatique :\n"
        "  - Audits de sécurité, tests d’intrusion\n"
        "  - Sécurisation des sites internet\n"
        "  - Sécurisation des données (conformité RGPD, correction des failles)\n"
        "  - Assistance à la création de VPN\n"
        "  - Prestation de sauvegarde des données\n\n"
        "Sources:\n"
        "[Document: FA_1.2.5_kap_numerique.pdf, Section: 3. DESCRIPTION TECHNIQUE, Page: 2, Mise à jour: Date non disponible]\n"
        "[Document: FAQs-Kap Numerique V2, Section: 3-Projets finançables, Page: 1, Mise à jour: 04/03/2025]\n\n"
        "Maintenant, réponds à la question en respectant ce format."
    )
    
    # Construction de l'invite utilisateur
    user_prompt = f"Question: {state['question']}\n\nContexte extrait des documents:\n{docs_content}"
    
    # Messages combinant instructions système et question de l'utilisateur
    messages = [
        {"role": "system", "content": system_instructions},
        {"role": "user", "content": user_prompt}
    ]
    
    # Appel au modèle LLM avec gestion des erreurs
    try:
        response = llm.invoke(messages)
        return {"answer": response.content}
    except Exception as e:
        return {"answer": f"Erreur lors de la génération de la réponse : {e}"}


# Construction du graphe d'application
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()



In [58]:
if __name__ == "__main__":
    print("Tapez 'exit' pour quitter.")
    while True:
        user_query = input("\nPosez votre question : ")
        if user_query.lower() in ["exit", "quit"]:
            break
        initial_state = {"question": user_query, "context": [], "answer": ""}
        state = graph.invoke(initial_state)  
        
        # Vous pouvez ici enrichir la réponse en affichant aussi la provenance des passages utilisés
        print("\nRéponse :", state["answer"])

Tapez 'exit' pour quitter.

Réponse : En réponse à votre question : "J'ai 11 salariés, quel est le montant maximal de ma subvention ?", voici les informations demandées :

- Votre entreprise étant composée de 11 salariés, elle entre dans la catégorie des entreprises de 10 à 19 salariés.
- Le montant maximal de la subvention pour les entreprises de cette taille est de 2 000 €.
- Cette subvention représente 50% des dépenses éligibles HT, avec un plafond de remboursement de 2 000 €.

Sources:
[Document: FA_1.2.5_kap_numerique.pdf, Section: 4. Montant de l'aide, Page: 4, Mise à jour: Date non disponible]
[Document: FA_1.2.5_kap_numerique.pdf, Section: 4. Montant de l'aide, Page: 6, Mise à jour: Date non disponible]

Réponse : En réponse à votre question : "Quels sont les codes APE inéligibles ?", voici les informations demandées :

- **Codes APE inéligibles dans le secteur numérique :**
  - 58.29A : Édition de logiciels système et de réseau
  - 58.29B : Édition de logiciels outils de dével